# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score

from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import ClusterCentroids 
from imblearn.combine import SMOTEENN

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

loan_size  interest_rate homeowner  borrower_income  debt_to_income  \
0    10700.0          7.672       own            52800        0.431818   
1     8400.0          6.692       own            43600        0.311927   
2     9000.0          6.963      rent            46100        0.349241   
3    10700.0          7.664       own            52700        0.430740   
4    10800.0          7.698  mortgage            53000        0.433962   

   num_of_accounts  derogatory_marks  total_debt loan_status  
0                5                 1       22800    low_risk  
1                3                 0       13600    low_risk  
2                3                 0       16100    low_risk  
3                5                 1       22700    low_risk  
4                5                 1       23000    low_risk

# Split the Data into Training and Testing

In [4]:
# Create our features
X = df.copy()
X.drop("loan_status", axis=1, inplace=True)

# Convert string to numbers
X = pd.get_dummies(X, drop_first=True) 

X.head()

loan_size  interest_rate  borrower_income  debt_to_income  num_of_accounts  \
0    10700.0          7.672            52800        0.431818                5   
1     8400.0          6.692            43600        0.311927                3   
2     9000.0          6.963            46100        0.349241                3   
3    10700.0          7.664            52700        0.430740                5   
4    10800.0          7.698            53000        0.433962                5   

   derogatory_marks  total_debt  homeowner_own  homeowner_rent  
0                 1       22800              1               0  
1                 0       13600              1               0  
2                 0       16100              0               1  
3                 1       22700              1               0  
4                 1       23000              0               0

In [5]:
# Create our target
y = df["loan_status"].to_frame()
y[:5] # showing first five to check

loan_status
0    low_risk
1    low_risk
2    low_risk
3    low_risk
4    low_risk

In [6]:
X.describe()

loan_size  interest_rate  borrower_income  debt_to_income  \
count  77536.000000   77536.000000     77536.000000    77536.000000   
mean    9805.562577       7.292333     49221.949804        0.377318   
std     2093.223153       0.889495      8371.635077        0.081519   
min     5000.000000       5.250000     30000.000000        0.000000   
25%     8700.000000       6.825000     44800.000000        0.330357   
50%     9500.000000       7.172000     48100.000000        0.376299   
75%    10400.000000       7.528000     51400.000000        0.416342   
max    23800.000000      13.235000    105200.000000        0.714829   

       num_of_accounts  derogatory_marks    total_debt  homeowner_own  \
count     77536.000000      77536.000000  77536.000000   77536.000000   
mean          3.826610          0.392308  19221.949804       0.398911   
std           1.904426          0.582086   8371.635077       0.489678   
min           0.000000          0.000000      0.000000       0.000000   
25%           3.000000          0.000000  14800.000000       0.000000   
50%           4.000000          0.000000  18100.000000       0.000000   
75%           4.000000          1.000000  21400.000000       1.000000   
max          16.000000          3.000000  75200.000000       1.000000   

       homeowner_rent  
count    77536.000000  
mean         0.103616  
std          0.304764  
min          0.000000  
25%          0.000000  
50%          0.000000  
75%          0.000000  
max          1.000000

In [7]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [8]:
# Create X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(58152, 9)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [9]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [10]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [11]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Check Scaled Data
X_train_scaled[:10]

array([[ 0.14045627,  0.1230916 ,  0.12859477,  0.32187276,  0.09115188,
         1.04495557,  0.12859477, -0.81861018, -0.3377677 ],
       [-0.28969527, -0.27393966, -0.27772578, -0.20875028, -0.43434275,
        -0.67471435, -0.27772578, -0.81861018, -0.3377677 ],
       [-0.48087373, -0.47526712, -0.48088605, -0.50399739, -0.43434275,
        -0.67471435, -0.48088605, -0.81861018, -0.3377677 ],
       [-0.43307912, -0.42015513, -0.42113303, -0.41487981, -0.43434275,
        -0.67471435, -0.42113303, -0.81861018, -0.3377677 ],
       [-0.62425758, -0.63160521, -0.63624391, -0.74520356, -0.43434275,
        -0.67471435, -0.63624391,  1.22158265, -0.3377677 ],
       [-0.09851681, -0.11872631, -0.12236792,  0.0029744 ,  0.09115188,
        -0.67471435, -0.12236792,  1.22158265, -0.3377677 ],
       [ 0.71399166,  0.72257505,  0.72612498,  0.98366993,  0.61664652,
         1.04495557,  0.72612498,  1.22158265, -0.3377677 ],
       [ 0.57060781,  0.59435532,  0.59466833,  0.84855097,  0

# Simple Logistic Regression

In [13]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [14]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9543211898288821

In [15]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  571,    54],
       [   93, 18666]])

In [16]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.91      1.00      0.89      0.95      0.90       625
   low_risk       1.00      1.00      0.91      1.00      0.95      0.92     18759

avg / total       0.99      0.99      0.92      0.99      0.95      0.92     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [17]:
# Resample the training data with the RandomOversampler
ros = RandomOverSampler(random_state=1)

# View the count of target classes with Counter
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)
Counter(y_resampled['loan_status'])


Counter({'low_risk': 56277, 'high_risk': 56277})

In [18]:
# Train the Logistic Regression model using the resampled data

model = LogisticRegression(solver='lbfgs', max_iter=100, random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [19]:
# Calculated the balanced accuracy score

y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9946414201183431

In [20]:
# Display the confusion matrix

confusion_matrix(y_test, y_pred)


array([[  622,     3],
       [  111, 18648]])

In [21]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [22]:
# Resample the training data with SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter

Counter(y_resampled['loan_status'])

Counter({'low_risk': 56277, 'high_risk': 56277})

In [23]:
# Train the Logistic Regression model using the resampled data

model = LogisticRegression(solver='lbfgs', random_state=1) 
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [24]:
# Calculated the balanced accuracy score

y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9946680739911509

In [25]:
# Display the confusion matrix

confusion_matrix(y_test, y_pred)


array([[  622,     3],
       [  110, 18649]])

In [26]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [27]:
# Resample the data using the ClusterCentroids resampler

cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter

Counter(y_resampled['loan_status'])

Counter({'high_risk': 1875, 'low_risk': 1875})

In [38]:
# Train the Logistic Regression model using the resampled data

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [41]:
# Calculate the balanced accuracy score

y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9932813049736127

In [43]:
# Display the confusion matrix

confusion_matrix(y_test, y_pred)

array([[  620,     5],
       [  102, 18657]])

In [44]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.99      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18759

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [45]:
# Resample the training data with SMOTEENN

sm = SMOTEENN(random_state=1)
X_resampled, y_resampled = sm.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter

Counter(y_resampled['loan_status'])

Counter({'high_risk': 55462, 'low_risk': 55906})

In [48]:
# Train the Logistic Regression model using the resampled data

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [49]:
# Calculate the balanced accuracy score

y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9946680739911509

In [50]:
# Display the confusion matrix

confusion_matrix(y_test, y_pred)

array([[  622,     3],
       [  110, 18649]])

In [51]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

**Simple Logistic Regression**
0.9543211898288821

**Oversampling**
0.9946414201183431


**SMOTE Oversampling**
0.9946680739911509

**Undersampling**
0.9932813049736127

**Combination (Over and Under) Sampling**
0.9946680739911509


**SMOTE Oversampling and Combination (Over and Under) Sampling had the best balanced accuracy scores.**

2. Which model had the best recall score?

**All models had the same recall score of 0.99. Additional decimal places would be required for further clarity.**

3. Which model had the best geometric mean score?

**All models had the same geometric mean score of 0.99. Additional decimal places would be required for further clarity.**
